<a href="https://colab.research.google.com/github/patjiang/ASICS-noise/blob/master/kmers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The idea is to slice protein databank structural files (pdb's) into k-long chains, and store the associated coordinates in cartesian space. The idea is that, visualization programs take chain index as the sequence itself, and form peptide bonds between those with the associated indices in PDB files, sothere is no extra information to take slices from.

tldr; use python_pdb to slice pdb files into dataframes, then store slices of k-mers as individual pdb files. Like epitope mapping, but with tertiary information for the kmers

questions:
do I need to align all of the pdb files to the origin? does it matter? or, can i just preserve the cartesian coordinates, and in a downstream processing step, use cartesian-independent information instead? (i.e. EDM, phi/psi angles)

In [ ]:
!pip install biopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 1.9 MB/s eta 0:00:00


In [ ]:
#@title testing scripts; src code/package-fied code ill be google colab independent
from google.colab import files


init_pdb = files.upload()
init_fname = list(init_pdb.keys())[0]

Saving 4g1q.pdb to 4g1q.pdb


In [ ]:
import pandas as pd
import copy
from biopandas.pdb import PandasPdb
import os
aa_dict = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

def norm_resnum(df):
  b = copy.deepcopy(df)
  if(list(b['residue_number'])[0] != 1):
    offset = list(b['residue_number'])[0]
    b['residue_number'] += (1 - offset)
  return b

def slice_pdb(df, idxs):
  out = pd.DataFrame({})
  for i in sorted(idxs):
    tmp_df = df.loc[df['residue_number'] == i]
    out = pd.concat([out, tmp_df], ignore_index=True)
  rlist = [aa_dict[i.upper()] for i in list(set(out['residue_name']))]
  rvals = ''.join(rlist)
  return out, rvals


def extract_kmer(k, input, out = None):
  if out == None:
    out = input.split('.')[0]
  print(f"Using: {out} as output filename head")

  ppdb = PandasPdb()
  ppdb.read_pdb('4g1q.pdb')
  a = ppdb.df['ATOM']
  chains = sorted(list(set(a['chain_id'])))
  b = a.loc[a['record_name'] != 'HETATM']
  outlist = {}
  for chain in chains:
    c = b.loc[b['chain_id'] == chain]
    d = norm_resnum(c)
    max_res_idx = max(d['residue_number'])
    for i in range(1, max_res_idx - k + 1):
      to_slice = [x for x in range(i, i + k)]
      k_mer, aas = slice_pdb(d, to_slice)
      outlist.update({f"{out}_res{aas}_ch{chain}_idx{i}-{i+k-1}": k_mer})
  return outlist, a

In [ ]:
kmer_list, ori = extract_kmer(3, init_fname)

Using: 4g1q as output filename head


In [ ]:
if not os.path.isdir('out_structs'):
  os.mkdir('out_structs')

for i in kmer_list.keys():
  tmp = PandasPdb()
  tmp._df = ({'ATOM':kmer_list[i]})
  try:
    tmp.to_pdb(path=f'./out_structs/{i}.pdb',
            records=['ATOM'],
            gz=False,
            append_newline=True)
  except:
    #sometimes pdb files are missing rows
    print("Failed: ",i)

Failed:  4g1q_res_chB_idx212-214
Failed:  4g1q_res_chB_idx213-215
Failed:  4g1q_res_chB_idx214-216
Failed:  4g1q_res_chB_idx215-217
Failed:  4g1q_res_chB_idx216-218
Failed:  4g1q_res_chB_idx217-219


In [ ]:
#@title Download kmers if you so choose
!zip -r outs.zip ./out_structs
files.download(f"outs.zip")